In [ ]:
from langchain_mistralai import ChatMistralAI
from pinecone import Pinecone as pc_client
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_mistralai import MistralAIEmbeddings
# from langchain.vectorstores import Pinecone as Pineconestore
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage
# from langchain.prompts import ChatPromptTemplate


# from pinecone import Pinecone as pc_client, ServerlessSpec


In [ ]:
from dotenv import load_dotenv
import os

# Load the .env file from a specific path
path = os.path.join(os.getcwd(),'.env')
load_dotenv(dotenv_path=path)



True

In [3]:
def read_doc(directory):
  file_object = PyPDFDirectoryLoader(directory)
  file = file_object.load()
  # print(file)
  return file

file = read_doc("D:/look_at_me/krish_pine_folder/krish_pine/papers")


In [4]:
len(file) , type(file)
for e in file[1]:
  print(e)


('id', None)
('metadata', {'source': 'D:\\look_at_me\\krish_pine_folder\\krish_pine\\papers\\Kim_Cluster_Self-Refinement_for_Enhanced_Online_Multi-Camera_People_Tracking_CVPRW_2024_paper.pdf', 'page': 1, 'page_label': '2'})
('page_content', 'Figure 2. Overview of our system’s architecture.\nonline MCPT.\nOnline MCPT, unlike offline MCPT, must predict the\ntracking results of the current frame solely based on ac-\ncumulated information from past frames. Therefore, if in-\ncorrect information is stored or tracking is done wrongly at\nany point, this misinformation can accumulate, leading to\nrepeated errors in the results. First, if inaccurate or low-\nquality appearance features are stored, it confuses predict-\ning the results of subsequent frames and makes accurate\ntracking difficult. The second issue arises when more than\none global ID is assigned to the same person. In situations\nwhere a new person enters a given location or an existing\nperson disappears, a new global ID can be 

In [5]:
def chunked(docs, chunk_size, chunk_overlap):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size,chunk_overlap = chunk_overlap)
  #  ["\n\n", "\n", " ", ""] by default splitting based on these
  chunks = text_splitter.split_documents(docs)
  return chunks

In [6]:
chunked_data = chunked(docs=file,chunk_size=100,chunk_overlap=20)
# for e in file[1]:
#   print(e)

In [7]:
len(chunked_data)
type(chunked_data)
chunked_data[0]

Document(metadata={'source': 'D:\\look_at_me\\krish_pine_folder\\krish_pine\\papers\\Kim_Cluster_Self-Refinement_for_Enhanced_Online_Multi-Camera_People_Tracking_CVPRW_2024_paper.pdf', 'page': 0, 'page_label': '1'}, page_content='Cluster Self-Refinement for Enhanced Online Multi-Camera People Tracking')

In [8]:
embeddings  = MistralAIEmbeddings(api_key=os.environ['MISTRAL_API_KEY'])

# testing_embeddings
string = "hi this is jai lets see the dimensions"
vector = embeddings.embed_query(string)
print(len(vector))

d:\look_at_me\krish_pine_folder\krish_pine\pineenv\Lib\site-packages\langchain_mistralai\embeddings.py:180: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


1024


In [53]:
type(file[1])
# file[7]
# file[0]
# len(file)
# for i in range(len(file)):
#   print(file[i].page_content)
chunked_data[0].page_content
# len(chunked_data)
# chunked_data

'Cluster Self-Refinement for Enhanced Online Multi-Camera People Tracking'

In [52]:
data = [{"id": f"vec{i+1}", "text": text.page_content} for i, text in enumerate(chunked_data)]

# data

In [67]:
only_content = [d['text'] for d in data]
# type(only_content)
embeddings_chunk = embeddings.embed_documents(only_content)
# li = [d['text'] for d in data]
# li
# len(data[0]['text'])
# emb = embeddings.embed_query(['hi'])


An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


HTTPStatusError: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429

In [ ]:
# Using exponential backoff
import time

def embed_with_retries(embeddings, texts, retries=5):
    for i in range(retries):
        try:
            return embeddings.embed_documents(texts)
        except Exception as e:
            if "429" in str(e):
                wait_time = 2 ** i  # Exponential backoff
                print(f"Rate limit hit. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

embeddings_chunk = embed_with_retries(embeddings, only_content)


An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


Rate limit hit. Retrying in 1 seconds...


In [ ]:
# embeddings_chunk
type(embeddings_chunk)
len(embeddings_chunk)

520

In [ ]:
index_name = "krish-vector"
pc = pc_client(api_key=os.environ['PINE'])
index = pc.Index(index_name) # Target the index where you'll store the vector embeddings

# print(pc,index)


In [77]:
#  Prepare the records for upsert
# Each contains an 'id', the embedding 'values', and the original text as 'metadata'
records = []
for d, e in zip(data, embeddings_chunk):
    records.append({
        "id": d['id'],
        "values": e,
        "metadata": {'text': d['text']}
    })

In [82]:
index.upsert(
    vectors=records,
    namespace="example-namespace"
)

{'upserted_count': 520}

In [91]:

def best_results(query):
    embedded_query = embeddings.embed_query(query)
    # embedded_query
    results = index.query(
        namespace="example-namespace",
        vector=embedded_query,
        top_k=3,
        include_values=False,
        include_metadata=True
    )
    return results

In [92]:
# type(results)
query = "what is multicamera people tracking?"
response = best_results(query)
texts = [match['metadata']['text'] for match in response['matches']]
print(texts)

['on Multi-Camera People Tracking (MCPT). MCPT presents', 'an online Multi-Camera People Tracking (MCPT) system', 'Figure 1. This is an example of Multi-Camera People Tracking. It']


In [ ]:
# results

{'matches': [{'id': 'vec5',
              'metadata': {'text': 'on Multi-Camera People Tracking (MCPT). '
                                   'MCPT presents'},
              'score': 0.839963,
              'values': []},
             {'id': 'vec152',
              'metadata': {'text': 'an online Multi-Camera People Tracking '
                                   '(MCPT) system'},
              'score': 0.834228694,
              'values': []},
             {'id': 'vec38',
              'metadata': {'text': 'Figure 1. This is an example of '
                                   'Multi-Camera People Tracking. It'},
              'score': 0.829051673,
              'values': []}],
 'namespace': 'example-namespace',
 'usage': {'read_units': 6}}

In [101]:
llm = ChatMistralAI(model_name="open-mistral-7b")
query = "who is narendra mode?"
def get_text(response):
  texts = [match['metadata']['text'] for match in response['matches']]
  return texts

relevant_ans_meta = best_results(query)
relevant_ans = get_text(relevant_ans_meta)
  
combined_input = ('my question is' + query +'here are some information that might be helpful \n\n '+ '\n\n'.join(ans for ans in relevant_ans)
+ "\n\nplease provide answer with the help of this relevant the inputs provided if answer is not availabe  in this query then return : sorry but no info available in the docs")

# print(combined_input)
messages = [
 SystemMessage(content='You are expert researcher in computer vision'),
 HumanMessage(content=combined_input)
]

response = llm.invoke(messages)
print(response.content)


Based on the provided information, it seems there are several authors with the name "Narendra" mentioned in the context of computer vision research. However, without more specific context or information, it's not possible to determine if any of these authors are referring to the same person named "Narendra Mode."

Here's a brief overview of the authors mentioned:

1. Meenakshi S. Rahman and other authors have published a paper titled "Multi-modal Sentiment Analysis on Social Media: A Review" in 2021.
2. Balaji Veeramani, John W. Raymond, and Pritam Chanda have published a paper titled "A Survey on Deep Learning for Computer Vision" in 2020.
3. Chengqi Lyu, Yining Li, and Kai Chen have published a paper titled "Rtmpose: Real-time Multi-pose Human Pose Estimation with Convolutional Neural Networks" in 2018.

Unfortunately, based on the information provided, it's not clear if any of these authors are the person you were asking about. If you have more specific information or context, I'd b